# The notebook cell at kaggle is this:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(len(filenames))
    for filename in filenames:
        print(f"Doing case:")
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Let's prepare everything to make it ready to run inference

In [ ]:
config_content = {
  "architecture": "STU-Net",
  "patch_size": [128, 128, 128],
  "device": "cuda",
  "batch_size": 1,
  "num_workers": 4,

  "infer_sw_batch_size": 2,
  "blend_mode": "gaussian",
  "deep_supervision": False,
  "activation": True,

  "infer_overlap": 0.5,
  "TTA": True,
  "TH": 0.5,

  "checkpoint_path": "/kaggle/input/focal-tversky-dasafe-epoch-150/pytorch/default/1/model_epoch_150.pth", 
  "dataset_path_imgs": "/kaggle/input",
  "pred_save_dir": "/kaggle/working/"
}


In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Any

class BaseInfer(ABC):
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.model = self._build_model()
        self.criterion = self._set_evaluation_criterion()


    @abstractmethod
    def _build_model(self) -> Any:
        """Initialize the neural network architecture. Load pretrained weights if necessary."""
        pass

    @abstractmethod
    def _set_evaluation_criterion(self) -> Any:
        """Define the evaluation function."""
        pass

    @abstractmethod
    def infer(self, input, **kwargs) -> Any:
        """Logic for inference. Returns predictions."""
        pass

    @abstractmethod
    def evaluate(self, predictions, gt, **kwargs) -> Any:
        """
        Logic for evaluation. Takes predictions and ground truth as input.
        Returns a dictionary of metrics. 
        Expected to be used after infer.
        """
        pass

In [ ]:
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid

class BasicResBlock(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super().__init__()
        
        if isinstance(stride, int):
            self.stride = (stride, stride, stride)
        else:
            self.stride = stride

        self.conv1 = nn.Conv3d(in_c, out_c, kernel_size=3, stride=self.stride, padding=1)
        self.norm1 = nn.InstanceNorm3d(out_c, affine=True, track_running_stats=False)
        self.act1 = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        
        self.conv2 = nn.Conv3d(out_c, out_c, kernel_size=3, stride=1, padding=1)
        self.norm2 = nn.InstanceNorm3d(out_c, affine=True, track_running_stats=False)
        self.act2 = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        
        self.conv3 = None
        if self.stride != (1,1,1) or in_c != out_c:
            self.conv3 = nn.Conv3d(in_c, out_c, kernel_size=1, stride=self.stride, bias=True)

    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.act1(out)
        
        out = self.conv2(out)
        out = self.norm2(out)
        
        if self.conv3 is not None:
            residual = self.conv3(x)
        
        out += residual
        
        # --- FIX: Activation MUST happen after addition ---
        out = self.act2(out) 
        
        return out

class Upsample_Layer_nearest(nn.Module):
    def __init__(self, in_c, out_c, scale_factor=(2,2,2)):
        super().__init__()
        self.scale_factor = scale_factor
        self.conv = nn.Conv3d(in_c, out_c, kernel_size=1, stride=1, bias=True)

    def forward(self, x):
        x = torch.nn.functional.interpolate(x, scale_factor=self.scale_factor, mode='nearest')
        return self.conv(x)

class STUNetReconstruction(nn.Module):
    def __init__(self):
        super().__init__()
        
        # --- ENCODER ---
        self.conv_blocks_context = nn.ModuleList([
            nn.Sequential(BasicResBlock(1, 64, stride=1), BasicResBlock(64, 64)),
            nn.Sequential(BasicResBlock(64, 128, stride=2), BasicResBlock(128, 128)),
            nn.Sequential(BasicResBlock(128, 256, stride=2), BasicResBlock(256, 256)),
            nn.Sequential(BasicResBlock(256, 512, stride=2), BasicResBlock(512, 512)),
            nn.Sequential(BasicResBlock(512, 1024, stride=2), BasicResBlock(1024, 1024)),
            nn.Sequential(BasicResBlock(1024, 1024, stride=(1,1,2)), BasicResBlock(1024, 1024)),
        ])

        # --- DECODER ---
        self.upsample_layers = nn.ModuleList([
            Upsample_Layer_nearest(1024, 1024, scale_factor=(1,1,2)), 
            Upsample_Layer_nearest(1024, 512, scale_factor=(2,2,2)),
            Upsample_Layer_nearest(512, 256, scale_factor=(2,2,2)),
            Upsample_Layer_nearest(256, 128, scale_factor=(2,2,2)),
            Upsample_Layer_nearest(128, 64, scale_factor=(2,2,2)),
        ])
        
        self.conv_blocks_localization = nn.ModuleList([
            nn.Sequential(BasicResBlock(2048, 1024), BasicResBlock(1024, 1024)),
            nn.Sequential(BasicResBlock(1024, 512), BasicResBlock(512, 512)),
            nn.Sequential(BasicResBlock(512, 256), BasicResBlock(256, 256)),
            nn.Sequential(BasicResBlock(256, 128), BasicResBlock(128, 128)),
            nn.Sequential(BasicResBlock(128, 64), BasicResBlock(64, 64)),
        ])

        # --- OUTPUT ---
        self.seg_outputs = nn.ModuleList([
            nn.Identity(), nn.Identity(), nn.Identity(), nn.Identity(),
            nn.Conv3d(64, 1, kernel_size=1, stride=1) 
        ])

    def forward(self, x):
        skips = []
        for i, block in enumerate(self.conv_blocks_context):
            x = block(x)
            if i < len(self.conv_blocks_context) - 1:
                skips.append(x)
                
        for i in range(len(self.upsample_layers)):
            x = self.upsample_layers[i](x)
            skip = skips[-(i+1)]
            if x.shape[2:] != skip.shape[2:]:
                x = torch.nn.functional.interpolate(x, size=skip.shape[2:], mode='nearest')
            x = torch.cat((x, skip), dim=1)
            x = self.conv_blocks_localization[i](x)

        return self.seg_outputs[4](x)


class STUNetSegmentation(nn.Module):
    def __init__(self, deep_supervision=True, activation=False):
        super().__init__()
        self.deep_supervision = deep_supervision
        self.activation = activation
        # --- ENCODER ---
        self.conv_blocks_context = nn.ModuleList([
            nn.Sequential(BasicResBlock(1, 64, stride=1), BasicResBlock(64, 64)),
            nn.Sequential(BasicResBlock(64, 128, stride=2), BasicResBlock(128, 128)),
            nn.Sequential(BasicResBlock(128, 256, stride=2), BasicResBlock(256, 256)),
            nn.Sequential(BasicResBlock(256, 512, stride=2), BasicResBlock(512, 512)),
            nn.Sequential(BasicResBlock(512, 1024, stride=2), BasicResBlock(1024, 1024)),
            nn.Sequential(BasicResBlock(1024, 1024, stride=(1,1,2)), BasicResBlock(1024, 1024)),
        ])

        # --- DECODER ---
        self.upsample_layers = nn.ModuleList([
            Upsample_Layer_nearest(1024, 1024, scale_factor=(1,1,2)), 
            Upsample_Layer_nearest(1024, 512, scale_factor=(2,2,2)),
            Upsample_Layer_nearest(512, 256, scale_factor=(2,2,2)),
            Upsample_Layer_nearest(256, 128, scale_factor=(2,2,2)),
            Upsample_Layer_nearest(128, 64, scale_factor=(2,2,2)),
        ])
        
        self.conv_blocks_localization = nn.ModuleList([
            nn.Sequential(BasicResBlock(2048, 1024), BasicResBlock(1024, 1024)),
            nn.Sequential(BasicResBlock(1024, 512), BasicResBlock(512, 512)),
            nn.Sequential(BasicResBlock(512, 256), BasicResBlock(256, 256)),
            nn.Sequential(BasicResBlock(256, 128), BasicResBlock(128, 128)),
            nn.Sequential(BasicResBlock(128, 64), BasicResBlock(64, 64)),
        ])

        # --- OUTPUT ---
        if deep_supervision:
            print(f"DOING DEEP SUPERVISION!")
            self.seg_outputs = nn.ModuleList([
                #nn.Conv3d(1024, 1, kernel_size=1, stride=1), 
                #nn.Conv3d(512, 1, kernel_size=1, stride=1), 
                None, # ignoring the deepest two outputs for memory efficiency
                None, # not using these outputs for deep supervision
                nn.Conv3d(256, 1, kernel_size=1, stride=1), 
                nn.Conv3d(128, 1, kernel_size=1, stride=1),
                nn.Conv3d(64, 1, kernel_size=1, stride=1) 
            ])
        else:
            self.seg_outputs = nn.ModuleList([
                #nn.Conv3d(1024, 1, kernel_size=1, stride=1), 
                #nn.Conv3d(512, 1, kernel_size=1, stride=1), 
                None, # ignoring the deepest two outputs for memory efficiency
                None, # not using these outputs for deep supervision
                None, 
                None,
                nn.Conv3d(64, 1, kernel_size=1, stride=1) 
            ])

    def forward(self, x):
        skips = []
        for i, block in enumerate(self.conv_blocks_context):
            x = block(x)
            if i < len(self.conv_blocks_context) - 1:
                skips.append(x)
        
        deep_supervision_preds = []
        for i in range(len(self.upsample_layers)):
            x = self.upsample_layers[i](x)
            skip = skips[-(i+1)]
            if x.shape[2:] != skip.shape[2:]:
                x = torch.nn.functional.interpolate(x, size=skip.shape[2:], mode='nearest')
            x = torch.cat((x, skip), dim=1)
            x = self.conv_blocks_localization[i](x)
            if i>=2 and self.deep_supervision:  # Start collecting predictions from the 3rd decoder block
                deep_supervision_preds.append(self.seg_outputs[i](x))
        if not self.deep_supervision:
            deep_supervision_preds.append(self.seg_outputs[-1](x))
        if self.training:
            # Return all scales for Deep Supervision Loss
            # index 0 = low res, index -1 = high res
            return deep_supervision_preds
        else:
            # Return only the final high-res prediction for validation/inference
            if self.activation:
                return sigmoid(deep_supervision_preds[-1])
            else:
                return deep_supervision_preds[-1]


In [ ]:
import torch
import nibabel as nib
import numpy as np
from torch.nn.functional import sigmoid
from torch.amp import autocast

import monai
from monai.transforms.transform import MapTransform, Transform
from monai.data import CacheDataset
from monai.metrics import DiceHelper
from monai.losses import DiceLoss, TverskyLoss, FocalLoss
from monai.transforms import (
    Compose,
    CopyItemsd,
    LoadImaged, 
    ScaleIntensityRanged, 
    ResizeWithPadOrCropd, 
    RandCoarseDropoutd,
    EnsureTyped,
    EnsureChannelFirstd,
    Resized,
    RandSpatialCropSamplesd,
    RandCropByPosNegLabeld
)
from monai.inferers import SlidingWindowInferer

import tifffile  

import os
import glob
import pandas as pd
from tqdm import tqdm  # 🚀 Import tqdm
import itertools
from monai.transforms.transform import MapTransform, Transform

class GetROIMaskdd(MapTransform):
    """
    Create a ROI mask from the ground truth by setting to 0 the regions with ignore_mask_value.
    The ROI mask will have 1 where the ground truth is not equal to ignore_mask_value, and 0 elsewhere.
    """

    def __init__(self, keys, ignore_mask_value=2, new_key_names=None):
        self.keys = keys
        self.ignore_mask_value = ignore_mask_value
        self.new_key_names = new_key_names

    def __call__(self, data):
        for key, new_key in zip(self.keys, self.new_key_names):
            gt = data[key]
            roi_mask = (gt != self.ignore_mask_value).float()
            data[new_key] = roi_mask
        return data

class GetBinaryLabeld(MapTransform):
    def __init__(self, keys, ignore_mask_value=2):
        super().__init__(keys)
        self.ignore_mask_value = ignore_mask_value

    def __call__(self, data):
        d = dict(data)
        for key in self.key_iterator(d):
            val = d[key]
            # 1. Use a small epsilon for the ignore value check (safety)
            # This handles values like 1.999 or 2.0000153
            mask = (val > (self.ignore_mask_value - 0.1)) & (val < (self.ignore_mask_value + 0.1))
            val[mask] = 0
            
            # 2. FORCE the remaining values into strictly 0 or 1
            # Anything that isn't background (0) should be 1
            # This fixes the 1.0000153 issue
            val[val > 0.5] = 1.0
            val[val <= 0.5] = 0.0
            
            d[key] = val
        return d


class VesuviusInferer(BaseInfer):
    def __init__(self, config):
        self.config = config
        self.model = self._build_model()
        self.criterion = self._set_evaluation_criterion()
        self.val_transforms = self._set_val_transforms()
        self.test_transforms = self._set_test_transforms()
        self.sliding_window = self._set_sliding_window_inferer()

        self.axes_combinations = self._define_flip_tta(spatial_dims=(2, 3, 4))
        

    def _build_model(self):
        """Initialize the neural network architecture. Load pretrained weights if necessary."""
        print(f"Loading weights from: {self.config['checkpoint_path']}")
        
        # Initialize the new Segmentation Model (1 output channels)
        model = STUNetSegmentation(self.config['deep_supervision'], activation=self.config['activation'])
        
        # Load trained Weights (1 output channel)
        checkpoint_state_dict = torch.load(self.config['checkpoint_path'], map_location='cpu')
        checkpoint_state_dict = checkpoint_state_dict['model_weights']
        load_result = model.load_state_dict(checkpoint_state_dict, strict=True)
        
        # Move to GPU
        model = model.to(self.config['device'])
        return model

    def _set_evaluation_criterion(self):
        """Define the evaluation function."""
        val_metric = DiceHelper
        
        def compute_masked_val(pred, target, roi_mask):
            pred_masked = pred * roi_mask
            pred_masked = (pred_masked > 0.5).float()
            # check if empty
            if torch.sum(pred_masked) == 0:
                # If both pred and target are empty in the ROI, return Dice of 1.0
                if torch.sum(target * roi_mask) == 0:
                    return torch.tensor(1.0, device=target.device, dtype=torch.float32)
                else:
                    return torch.tensor(0.0, device=target.device, dtype=torch.float32)
            target_masked = target * roi_mask
            results = val_metric(include_background=True, sigmoid=False, softmax=False)(pred_masked, target_masked)
            return results
        return compute_masked_val
        
    def save_nifti(self, torch_tensor, save_path, real_file, **kwargs):
        """Logic for saving predictions to disk."""
        # Ensure tensor is 3D (H, W, D) or (H, W)
        data = torch_tensor.detach().cpu().numpy().astype('float32')
        while data.ndim > 3:
            data = np.squeeze(data, axis=0)

        if real_file is not None:
            # Load the real file to get metadata
            real_nii = nib.load(real_file)
            real_affine = real_nii.affine
            real_header = real_nii.header
            pred_nii = nib.Nifti1Image(data, affine=real_affine, header=real_header)
        else:
            # If no real file is provided, use identity affine
            affine = np.eye(4)
            pred_nii = nib.Nifti1Image(data, affine=affine)

        # Save the Nifti1Image to disk
        nib.save(pred_nii, save_path)
    
    def save_tiff(self, torch_tensor, save_path, **kwargs):
        """Logic for saving predictions to disk."""
        # Ensure tensor is 3D (H, W, D) or (H, W)
        data = torch_tensor.detach().cpu().numpy().astype('uint8')
        while data.ndim > 3:
            data = np.squeeze(data, axis=0)
        data = np.transpose(data, (2, 1, 0))
        
        # If the data is a probability map (0-1), 
        # many Vesuvius tools expect uint16 or uint8
        # Convert if necessary: (data * 255).astype('uint8')
        tifffile.imwrite(save_path, data)

    def _set_dataloader_transforms(self, gt_available, **kwargs):
        """Define the data loading and augmentation transforms."""

        if gt_available:
            general_keys = ["image", 'gt']
            all_keys = ["image", 'gt', 'roi_mask']
        else:
            general_keys = ["image"]
            all_keys = ["image"]

        transform_list = [
                LoadImaged(keys=general_keys),
                EnsureChannelFirstd(keys=general_keys),
                # Normalize uint8 input
                ScaleIntensityRanged(keys=["image"], a_min=0, a_max=255, b_min=0, b_max=1, clip=True),
            ]
        
        if gt_available:
            transform_list.append(GetROIMaskdd(keys=["gt"], ignore_mask_value=2, new_key_names=["roi_mask"]))
            transform_list.append(GetBinaryLabeld(keys=["gt"], ignore_mask_value=2))

        # Get random patches
        transform_list.append(EnsureTyped(keys=all_keys, track_meta=False))

        return transform_list
    
    def _set_val_transforms(self, **kwargs):
        """Define the validation data loading transforms."""
        transform_list = self._set_dataloader_transforms(gt_available=True, **kwargs)
        return Compose(transform_list)

    def _set_test_transforms(self, **kwargs):   
        """Define the test data loading transforms."""
        transform_list = self._set_dataloader_transforms(gt_available=False, **kwargs)
        return Compose(transform_list)

    def load_input_data(self, file_dict, transforms, **kwargs):
        '''
        Monai data loading logic. It must handle tiff files.
        '''
        transformed_data = transforms(file_dict)
        return transformed_data
    
    def _set_sliding_window_inferer(self, **kwargs):
        """Define the sliding window inferer."""
        
        inferer = monai.inferers.SlidingWindowInferer(
            roi_size=self.config['patch_size'],
            sw_batch_size=self.config['infer_sw_batch_size'],
            overlap=self.config['infer_overlap'],
            mode=self.config['blend_mode'],
            padding_mode='constant',
            cval=0.0,
            device=self.config['device'],
            progress=True
        )

        return inferer

    def _define_flip_tta(self, spatial_dims=(2, 3, 4)):
        """
        # All combinations possble
        axes_combinations = []
        for i in range(len(spatial_dims) + 1):
            axes_combinations.extend(itertools.combinations(spatial_dims, i))
        return axes_combinations
        """
        # ignore the dim=2 (z axis)
        return [(), (3,), (4,), (3, 4)]
    def infer(self, input, test=False, threshold=0.5, **kwargs):
        """Logic for inference. Returns predictions."""
        if test:
            data = self.load_input_data(file_dict=input, transforms=self.test_transforms)
        else:
            data = self.load_input_data(file_dict=input, transforms=self.val_transforms)

        input_image = data['image'].unsqueeze(0).to(self.config['device'])

        self.model.eval()
        
        # TTA
        if self.config["TTA"]:
            print(f"Doing inference to these axes combinations:")
            print(self.axes_combinations)
            all_predictions = []
            all_logits_pred = []
            for axes in self.axes_combinations:
                print(f"Doing axes: {axes}")
                if axes:
                    # torch.flip requires a tuple of dims
                    aug_input = torch.flip(input_image, dims=axes)
                else:
                    aug_input = input_image  # Original image

                with torch.no_grad():
                    with autocast(device_type=self.config['device']):
                        if self.config['activation']:
                            prediction = self.sliding_window(inputs=aug_input, network=self.model)
                        else: # in case sigmoid is not applied in the end of the network (recommended to be True!)
                            logits_pred = self.sliding_window(inputs=aug_input, network=self.model)
                            prediction = sigmoid(logits_pred)
                    
                # revert flip
                if axes:
                    pred_prob_aligned = torch.flip(prediction, dims=axes)
                    if not self.config['activation']:
                        logits_pred = torch.flip(logits_pred, dims=axes)
                else:
                    pred_prob_aligned = prediction

                all_predictions.append(pred_prob_aligned)
                if not self.config['activation']:
                    all_logits_pred.append(logits_pred)

            final_pred = torch.stack(all_predictions).mean(dim=0)
            if not self.config['activation']:
                all_logits_pred = torch.stack(all_logits_pred).mean(dim=0)
                
        else:
            if self.config['activation']:
                with torch.no_grad():
                    with autocast(device_type=self.config['device']):
                        final_pred = self.sliding_window(inputs=input_image, network=self.model)
            else:
                with torch.no_grad():
                    with autocast(device_type=self.config['device']):
                        logits_pred = self.sliding_window(inputs=input_image, network=self.model)
                        final_pred = sigmoid(logits_pred)
                        all_logits_pred = logits_pred
        # Binary seg
        final_pred[final_pred>threshold] = 1.0
        final_pred[final_pred<=threshold] = 0.0

        if test:
            if self.config['activation']:
                return None, final_pred
            else:
                return all_logits_pred, final_pred
        else:
            if self.config['activation']:
                return final_pred, data['gt'].unsqueeze(0).to(self.config['device']), data['roi_mask'].unsqueeze(0).to(self.config['device']) 
            else:
                return all_logits_pred, final_pred, data['gt'].unsqueeze(0).to(self.config['device']), data['roi_mask'].unsqueeze(0).to(self.config['device']) 

    def create_dfs(self, path_dir):
        # Generate DataFrame
        result_df = glob.glob(os.path.join(path_dir, '**/*.tif'), recursive=True)
        result_df = pd.DataFrame({'tif_paths': result_df})
        result_df['id'] = result_df['tif_paths'].apply(lambda x: os.path.basename(x).split('.')[0])

        # save dataframe to csv (take name from the path_dir last folder)
        csv_name = os.path.basename(os.path.normpath(path_dir)) + '_df.csv'
        result_df.to_csv(os.path.join(path_dir, csv_name), index=False)
        return result_df

    def evaluate(self, predictions, gt, roi_mask, **kwargs):
        """
        Logic for evaluation. Takes predictions and ground truth as input.
        Returns a dictionary of metrics. 
        Expected to be used after infer.
        """
        results = self.criterion(predictions, gt, roi_mask)
        return results
    
    def dataset_inference_save_logits(self, dataset_path, pred_save_dir):
        """ inference on all cases in a dataset directory and save predictions """
        os.makedirs(pred_save_dir, exist_ok=True)
        tif_files = glob.glob(os.path.join(dataset_path, '**/*.tif'), recursive=True)
        nii_files = glob.glob(os.path.join(dataset_path, '**/*.nii.gz'), recursive=True)

        all_cases = tif_files + nii_files

        print(f"Found cases: {all_cases}")
        
        # 🏁 Wrap the list in tqdm for a visual progress bar
        # 'desc' adds a label to the bar, 'unit' labels each iteration
        for case in tqdm(all_cases, desc="🌋 Running Vesuvius Inference", unit="vol"):
            # Optional: you can still print the case, but it might "flicker" the bar. 
            # Using tqdm.write() keeps the bar at the bottom.
            # tqdm.write(f"🔍 Processing: {os.path.basename(case)}")
            
            input_data = {
                'image': str(case),
                'gt': None
            }
            
            logits_pred, pred = self.infer(input_data, test=True)
            
            # 💾 Robust filename extraction
            file_name = os.path.basename(case).replace('.tif', '')
            file_name = os.path.basename(case).replace('.nii.gz', '')
            save_path = os.path.join(pred_save_dir, f"{file_name}.nii.gz")
            
            self.save_nifti(
                torch_tensor=logits_pred, 
                save_path=save_path, 
                real_file=str(case)
            )

        print(f"✅ Inference completed. Predictions saved to: {pred_save_dir}")

    def dataset_inference(self, dataset_path, pred_save_dir):
        """ inference on all cases in a dataset directory and save predictions """
        os.makedirs(pred_save_dir, exist_ok=True)
        all_cases = glob.glob(os.path.join(dataset_path, '**/*.tif'), recursive=True)
        
        # 🏁 Wrap the list in tqdm for a visual progress bar
        # 'desc' adds a label to the bar, 'unit' labels each iteration
        for case in tqdm(all_cases, desc="🌋 Running Vesuvius Inference", unit="vol"):
            # Optional: you can still print the case, but it might "flicker" the bar. 
            # Using tqdm.write() keeps the bar at the bottom.
            # tqdm.write(f"🔍 Processing: {os.path.basename(case)}")
            
            input_data = {
                'image': str(case),
                'gt': None
            }

            # 💾 Robust filename extraction
            file_name = os.path.basename(case).replace('.tif', '')
            save_path = os.path.join(pred_save_dir, f"{file_name}.tif")
            if os.path.isfile(save_path):
                print(f"Case already predicted: {save_path}")
            else:
                # Inference
                logits_pred, pred = self.infer(input_data, test=True, threshold=self.config["TH"])
            
            self.save_tiff(
                torch_tensor=pred, 
                save_path=save_path
            )

        # 📊 Creating the dataframe for testing
        print("\n📝 Generating submission dataframes...")
        self.create_dfs(pred_save_dir)
        print(f"✅ Inference completed. Predictions saved to: {pred_save_dir}")

In [ ]:
import os
import json
import sys


# Competition
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

###################################################
################ Inference process ################
print(f"🚀 Starting inference on dataset: {config_content['dataset_path_imgs']}")
print("📋 Using the following configuration:")
for key, value in config_content.items():
    print(f"  🔹 {key}: {value}")

# 🧪 Initialize Inference Object
infer_object = VesuviusInferer(config_content)
pred_save_dir = config_content["pred_save_dir"]

print("=" * 50)

for dirname, _, filenames in os.walk(config_content["dataset_path_imgs"]):
    for filename in filenames:
        case = os.path.join(dirname, filename)
        print(f"Doing case: {case}")
        
        input_data = {
                'image': str(case),
                'gt': None
            }
        
        logits_pred, pred = infer_object.infer(input_data, test=True, threshold=config_content["TH"])

        # 💾 Robust filename extraction
        file_name = os.path.basename(case).replace('.tif', '')
        save_path = os.path.join(pred_save_dir, f"{file_name}.tif")
        infer_object.save_tiff(
                        torch_tensor=pred, 
                        save_path=save_path
                    )
        print(f"✅ Inference completed. Predictions saved to: {pred_save_dir}")

print("🎉 Results saved! 🏆")




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

